In [ ]:
!pip install trubrics

**In this tutorial of the [Titanic Use Case](https://www.kaggle.com/c/titanic), you will:**
- Initialise a `DataContext` with ML datasets and metadata from the titanic use case
- Build some out-of-the-box validations on a trained model and the `DataContext` with the `ModelValidator`:
    - Minimum functionality validation
    - Performance validations:
        - With sklearn metrics
        - With custom metrics
    - Explainability validation (with permutation importance)
    - Inference time validation
- Example of how feedback can be used to create meaningful validations
- Build a custom validation
- Save validations to a `Trubric`

## Load data & model

In [ ]:
from trubrics.example import get_titanic_data_and_model
train_df, test_df, model = get_titanic_data_and_model()

## Init DataContext

In [ ]:
from trubrics.context import DataContext

In [ ]:
data_context = DataContext(
    name="my_first_dataset",
    version=0.1,
    testing_data=test_df,
    target="Survived",
    training_data=train_df,
    minimum_functionality_data=test_df.head(),
)

## Init ModelValidator

In [ ]:
from trubrics.validations import ModelValidator
from examples.classification_titanic.custom_scorer import custom_scorers  # see .py script for custom_scorer example
from examples.classification_titanic.slicing_functions import slicing_functions  # see .py script for slicing_functions examples

In [ ]:
model_validator = ModelValidator(
    data=data_context, model=model, custom_scorers=custom_scorers, slicing_functions=slicing_functions
)

## Use the ModelValidator to build out-of-the-box validations

In [ ]:
import rich
minimum_functionality = [
    model_validator.validate_minimum_functionality(severity="warning"),  # validation severity can be passed in to any validation
]
_ = [rich.print(val.dict()) for val in minimum_functionality]

In [ ]:
performance = [
    model_validator.validate_performance_against_threshold(metric="my_custom_loss", threshold=-0.7, severity="experiment"),
    model_validator.validate_performance_between_train_and_test(metric="my_custom_loss", threshold=0.2),
    model_validator.validate_performance_std_across_slices(metric="accuracy", dataset="testing_data", data_slices=["male", "children"], std_threshold=0.07),
]
_ = [rich.print(val.dict()) for val in performance]

In [ ]:
explainability = [
    model_validator.validate_feature_in_top_n_important_features(dataset="testing_data", feature="Sex", top_n_features=3),
]
_ = [rich.print(val.dict()) for val in explainability]

In [ ]:
import pandas as pd
# recap of calculated model performance metrics by validations
pd.DataFrame(model_validator.performances)

## Build custom validations

In [ ]:
from examples.classification_titanic.custom_validator import CustomValidator  # see script for CustomValidator example

In [ ]:
model_custom_validator = CustomValidator(data=data_context, model=model, custom_scorers=custom_scorers)

In [ ]:
custom = [model_custom_validator.validate_performance_for_different_fares(fare_cutoff=25, severity="warning")]
_ = [rich.print(val.dict()) for val in custom]

## Save validations as a trubric

In [ ]:
from trubrics.validations import Trubric

validations = minimum_functionality + performance + explainability + custom

trubric = Trubric(
    trubric_name="my_first_trubric",
    model_name="my_model",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    metadata={"tag": "master"},
    validations=validations,
)

In [ ]:
# save trubric to a local .json
trubric.save_local(path=".")

In [ ]:
!cat my_first_trubric.json

## Execute the trubric from the CLI tool

Example code snippet:

```bash
(venv)$ trubrics run \
        --trubric-config-path "examples/classification_titanic" \
        --trubric-output-file-path "examples/classification_titanic" \
        --trubric-output-file-name "my_new_trubric.json"
```